In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [3]:
T = 3.
K = 100.
S0 = 100.
sigma = 0.2
p = 5
M = np.eye(p)
r = 0.05
mu = 0.05
R = 0.05
Q = 0.1

In [4]:
M_run = 20
m = 8
N = 10000
n_picard = 3
n_estimators=100
learning_rate=0.1
max_depth=1
random_state=0
loss="ls"

In [5]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test = BsdeHD(T, K, M, mu, Q, sigma, S0, r, R)
    a[i] = test.get_price(N, m, option_payoff='max',
                          oType='European',n_picard=n_picard, regression='gbr')
    elapsed = time.time() - start
    timing[i] = round(elapsed, 3)
    print (a[i])
    

22.6828176466
23.3121700609
23.2619004607
22.7951352819
23.1562221602
22.999177403
23.1931949896
23.0094775065
22.9027083353
23.1479977897
22.9347412783
23.1703995755
23.4449674367
23.1403311441
23.397779214
23.1878497913
22.8178668797
22.847140383
22.8767978079
23.0465090618


In [7]:
a.mean()

23.066259210333833

In [8]:
import pandas as pd

In [9]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [10]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard', 'Number of trees', 'Learning rate', 'max_depth']

svalues = [N, m, M_run, n_picard, n_estimators, learning_rate, max_depth, random_state]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,10000.0
1,m_discretization,8.0
2,N_run,20.0
3,n_picard,3.0
4,Number of trees,100.0
5,Learning rate,0.1
6,max_depth,1.0


In [11]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [12]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [13]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,23.0663
1,std,0.2093
2,95% confidence interval,"[23.0458, 23.0868]"
3,min,22.6828
4,max,23.445


In [14]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_GradBoost.csv')